In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
# set the working directory to the root of RCANE project
os.chdir('/Volumes/gech/cna/RCANE')

In [ ]:
# load the ground truth data and the predicted data
ground_truth = np.load('data/predict/TCGA_test.npz', allow_pickle=True)
prediction = np.load('data/predict/TCGA_test_prediction.npz', allow_pickle=True)